In [2]:
import requests
from requests import Response
from pydantic import BaseModel,Field,RootModel,field_validator
import pandas as pd

class Site(BaseModel):
    站名:str = Field(alias='sna')
    地區:str = Field(alias = 'sarea')
    時間:str = Field(alias = 'mday')
    地點:str = Field(alias = 'ar')
    總車輛數:int = Field(alias='total')
    經度: float = Field(alias='longitude')
    緯度: float = Field(alias='latitude')    
    可借:int = Field(alias='available_rent_bikes')
    可還:int = Field(alias='available_return_bikes')
    狀態:bool = Field(alias='act')
    
    @field_validator("站名")
    @classmethod
    def removefilter(cls,value:str)->str:
        return value.split("_")[-1]
    
class YouBike(RootModel):
    root: list[Site]
    def __iter__(self):
        return iter(self.root)

    def __getitem__(self, item):
        return self.root[item]

try:

    youbike_url = "https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json"
    res:Response = requests.get(youbike_url)
    if res.status_code == 200:
        print("Download Success")
    else:
        print("Download Failure") 
        res.raise_for_status()
except Exception as e:
    print(e)
else:  
    data = YouBike.model_validate_json(res.text)
    all_sites = data.model_dump()
    
df1 = pd.DataFrame(all_sites)

       

Download Success


樞紐分析
pivot
stack ,unstack

In [3]:
df2 = df1.groupby(by='地區')[['總車輛數','可借','可還']].agg([('車輛數','sum'),('站點數','count')])
s1 = df2.stack(level=[0,1])
s1.index.names = ['行政區域','主題','數量']
s1.unstack(level=['數量'])


/tmp/ipykernel_46334/3705981719.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  s1 = df2.stack(level=[0,1])


數量            車輛數  站點數
行政區域   主題             
中山區    可借    1558  156
       可還    2227  156
       總車輛數  3819  156
中正區    可借    1533  121
       可還    1649  121
       總車輛數  3209  121
信義區    可借    1260  106
       可還    1966  106
       總車輛數  3331  106
內湖區    可借    1432  168
       可還    2030  168
       總車輛數  3499  168
北投區    可借     912   94
       可還    1530   94
       總車輛數  2472   94
南港區    可借    1110   92
       可還    1184   92
       總車輛數  2317   92
士林區    可借    1008  127
       可還    1847  127
       總車輛數  2937  127
大同區    可借     585   62
       可還    1030   62
       總車輛數  1629   62
大安區    可借    2091  185
       可還    2792  185
       總車輛數  4987  185
文山區    可借     842   97
       可還    1358   97
       總車輛數  2203   97
松山區    可借     876   88
       可還    1449   88
       總車輛數  2343   88
臺大公館校區 可借     539   53
       可還     863   53
       總車輛數  1437   53
萬華區    可借     969   80
       可還    1059   80
       總車輛數  2057   80

pandas.cut 需要指定範圍